# JavaScript 逆向爬虫实战：[Scrape Center](https://scrape.center/) - [spa6](https://spa6.scrape.center/)

## 导入依赖库

In [1]:
import base64
import hashlib
import random
import time
from typing import List, Any

import requests
import urllib3

## 禁用安全请求警告

In [2]:
urllib3.disable_warnings()

## 定义常量

- INDEX_URL：电影列表 URL
    - 参数
        - limit：电影数量
        - offset：偏移量
        - token：列表页 token
- DETAIL_URL：电影详情 URL
    - 参数
        - id：电影 ID（加密后）
        - token：详情页 token
- SECRET：密钥

In [3]:
INDEX_URL = 'https://spa6.scrape.center/api/movie?limit={limit}&offset={offset}&token={token}'
DETAIL_URL = 'https://spa6.scrape.center/api/movie/{id}?token={token}'
SECRET = 'ef34#teuq0btua#(-57w1q5o5--j@98xygimlyfxs*-!i-0-mb'

## 获取 token

- 参数：path（URL 中 "/" 至 "?" 之间的部分，例如：https://spa6.scrape.center **/api/movie** /?limit=10&offset=0&token=NjY2N...E5）
- 返回：token（base64 编码后的字符串）

In [58]:
def get_token(args: List[Any]):
    # 获取时间戳
    timestamp = str(int(time.time()))
    # 将时间戳加入参数列表
    args.append(timestamp)
    sign1 = ','.join(args)
    print('1. 拼接 path 和时间戳：\t\t', sign1)
    # 将参数列表转为字符串并进行SHA1加密
    sign2 = hashlib.sha1(sign1.encode('utf-8')).hexdigest()
    print('2. 对上一步结果进行 SHA1 加密：\t', sign2)
    # 将加密后的字符串和时间戳拼接并进行base64编码
    sign3 = sign2 + ',' + timestamp
    print('3. 拼接加密后的字符串和时间戳：\t', sign3)
    token = base64.b64encode(sign3.encode('utf-8')).decode('utf-8')
    print('4. 对上一步结果进行 base64 编码：', token, '\n')
    return token


get_token(args=['/api/movie'])

1. 拼接 path 和时间戳：		 /api/movie,1684751081
2. 对上一步结果进行 SHA1 加密：	 85861d23b023802d73e0dfe5673ebca21e82105e
3. 拼接加密后的字符串和时间戳：	 85861d23b023802d73e0dfe5673ebca21e82105e,1684751081
4. 对上一步结果进行 base64 编码： ODU4NjFkMjNiMDIzODAyZDczZTBkZmU1NjczZWJjYTIxZTgyMTA1ZSwxNjg0NzUxMDgx 



'ODU4NjFkMjNiMDIzODAyZDczZTBkZmU1NjczZWJjYTIxZTgyMTA1ZSwxNjg0NzUxMDgx'

## 获取电影列表

- 参数：无
- 返回：JSON 格式的电影列表

首先，我们需要获取列表中电影的总数 count，然后根据 limit=count 构造新的 URL，最后获取全部电影列表。

In [59]:
def get_mov_list():
    # 获取token
    token = get_token(args=['/api/movie'])
    # 构造URL
    url = INDEX_URL.format(limit=10, offset=0, token=token)
    # 获取电影总数
    mov_count = requests.get(url, verify=False).json()['count']
    # 根据电影总数构造新的URL
    url = INDEX_URL.format(limit=mov_count, offset=0, token=token)
    # 获取电影列表
    mov_list = requests.get(url, verify=False).json()
    return mov_list

get_mov_list()

1. 拼接 path 和时间戳：		 /api/movie,1684751087
2. 对上一步结果进行 SHA1 加密：	 b0b94b33fa9b979f8262d022777210405b1c598b
3. 拼接加密后的字符串和时间戳：	 b0b94b33fa9b979f8262d022777210405b1c598b,1684751087
4. 对上一步结果进行 base64 编码： YjBiOTRiMzNmYTliOTc5ZjgyNjJkMDIyNzc3MjEwNDA1YjFjNTk4YiwxNjg0NzUxMDg3 



{'count': 101,
 'results': [{'id': 1,
   'name': '霸王别姬',
   'alias': 'Farewell My Concubine',
   'cover': 'https://p0.meituan.net/movie/ce4da3e03e655b5b88ed31b5cd7896cf62472.jpg@464w_644h_1e_1c',
   'categories': ['剧情', '爱情'],
   'published_at': '1993-07-26',
   'minute': 171,
   'score': 9.5,
   'regions': ['中国内地', '中国香港']},
  {'id': 2,
   'name': '这个杀手不太冷',
   'alias': 'Léon',
   'cover': 'https://p1.meituan.net/movie/6bea9af4524dfbd0b668eaa7e187c3df767253.jpg@464w_644h_1e_1c',
   'categories': ['剧情', '动作', '犯罪'],
   'published_at': '1994-09-14',
   'minute': 110,
   'score': 9.5,
   'regions': ['法国']},
  {'id': 3,
   'name': '肖申克的救赎',
   'alias': 'The Shawshank Redemption',
   'cover': 'https://p0.meituan.net/movie/283292171619cdfd5b240c8fd093f1eb255670.jpg@464w_644h_1e_1c',
   'categories': ['剧情', '犯罪'],
   'published_at': '1994-09-10',
   'minute': 142,
   'score': 9.5,
   'regions': ['美国']},
  {'id': 4,
   'name': '泰坦尼克号',
   'alias': 'Titanic',
   'cover': 'https://p1.meituan.ne

## 获取电影详情

In [43]:
def get_mov_detail(mov_id):
    # 对电影ID进行加密
    encrypt_id = base64.b64encode((SECRET + str(mov_id)).encode('utf-8')).decode('utf-8')
    # 构造URL
    url = DETAIL_URL.format(id=encrypt_id, token=get_token(args=[f'/api/movie/{encrypt_id}']))
    # 获取电影详情
    return requests.get(url, verify=False).json()

## 获取所有电影详情

In [44]:
def get_all_mov_detail(mov_list):
    # 获取所有电影详情
    all_mov_detail = [get_mov_detail(mov['id']) for mov in mov_list['results']]
    # 打印获取电影详情的数量
    print(f'已成功获取 {len(all_mov_detail)} 部电影详情')
    return all_mov_detail

In [55]:
# 生成1-100的随机数
rid = random.randint(1, 100)
m_info = get_mov_detail(mov_id=rid)

print('电影', rid, '详情：')
for key, value in m_info.items():
    print(key + ": ", value)

1. 拼接 path 和时间戳：		 /api/movie/ZWYzNCN0ZXVxMGJ0dWEjKC01N3cxcTVvNS0takA5OHh5Z2ltbHlmeHMqLSFpLTAtbWI1Mw==,1684750893
2. 对上一步结果进行 SHA1 加密：	 409ba734eb65b35c56344e71e767eec725015e4b
3. 拼接加密后的字符串和时间戳：	 409ba734eb65b35c56344e71e767eec725015e4b,1684750893
4. 对上一步结果进行 base64 编码： NDA5YmE3MzRlYjY1YjM1YzU2MzQ0ZTcxZTc2N2VlYzcyNTAxNWU0YiwxNjg0NzUwODkz 



{'id': 53,
 'name': '蝙蝠侠：黑暗骑士崛起',
 'alias': 'The Dark Knight Rises',
 'cover': 'https://p0.meituan.net/movie/f7f4b4099773268f8290ed033f49dc01377512.jpg@464w_644h_1e_1c',
 'categories': ['剧情', '动作', '科幻', '惊悚', '犯罪'],
 'regions': ['美国', '英国'],
 'actors': [{'name': '克里斯蒂安·贝尔',
   'role': '布鲁斯·韦恩,蝙蝠侠 The Batman',
   'image': 'https://p1.meituan.net/moviemachine/9a631b74966202bd23cf47c7d36c157d44740.jpg@128w_170h_1e_1c'},
  {'name': '迈克尔·凯恩',
   'role': '阿尔弗莱德 Alfred',
   'image': 'https://p0.meituan.net/movie/455ed426d5ea2fb7a31a80d6c4eebdfa53569.jpg@128w_170h_1e_1c'},
  {'name': '加里·奥德曼',
   'role': '吉姆·戈登 Jim Gordon',
   'image': 'https://p0.meituan.net/movie/9f1945dbc62b6959cba7e2d44b09fb94108912.jpg@128w_170h_1e_1c'},
  {'name': '安妮·海瑟薇',
   'role': '瑟琳娜·凯尔,猫女 Selina Kyle,Catwoman',
   'image': 'https://p0.meituan.net/moviemachine/ac5161756bb66857875275b2f445451f171963.jpg@128w_170h_1e_1c'},
  {'name': '汤姆·哈迪',
   'role': '贝恩 Bane',
   'image': 'https://p1.meituan.net/movie/2d2d29aed8